In [1]:
import requests
from bs4 import BeautifulSoup

import re

from tqdm import tqdm
import time

import pandas as pd

Scrapping poetry of given creator from the [poezja.org](https://poezja.org/) website

In [12]:
p_list = [#'Kaczmarski_Jacek', 'Szymborska_Wisława', 
          #'Kochanowski_Jan', 
          'Tuwim_Julian']

In [13]:
for p in p_list:
    path = f'https://poezja.org/wz/{p}/#author-works'
    
    getpage = requests.get(path)
    
    soup = BeautifulSoup(getpage.text, 'html.parser')
    
    poem_list = []
    title_list = []
    
    for a in tqdm( soup.find_all(id='allPoems')[0].find_all("a") ):
        poem_path = a['href']

        get_poem = requests.get(poem_path)

        poem_soup = BeautifulSoup(get_poem.text, 'html.parser')

        poem = poem_soup.find_all("div", class_ = "container")

        poem_txt = poem[2].find_all("p")[0].text

        clean_txt = re.sub('Czytaj dalej.*', '', poem_txt).strip()
        clean_txt = re.sub('#.*', '', clean_txt).strip()

        poem_list.append(clean_txt)
        
        title = poem_soup.find_all("div", class_ = "col-12 col-lg-8")[0].find_all("h1")[0].text
        
        title_list.append(title)
        
        time.sleep(0.5)
    
    df = pd.DataFrame({'title' : title_list, 'poem' : poem_list})
    
    df.to_csv(f'poems/{p.lower()}.csv', sep=';', index=False)

100%|█████████████████████████████████████████| 443/443 [05:27<00:00,  1.35it/s]
